# Lab 2: Pipeline Interativo de Recomendação de Frequência

Este notebook executa o pipeline de recomendação de frequência de ponta a ponta. As células abaixo irão compilar o código MapReduce em Java, ingerir dados do PostgreSQL com Sqoop, executar o job no Hadoop e verificar os resultados.

## 1. Compilando a Aplicação MapReduce

A primeira etapa é compilar nosso código-fonte Java em um arquivo `.jar` executável. A célula abaixo usa o Maven para isso. Ela navega até o diretório do projeto e executa o `mvn package`. A flag `-q` é para uma saída mais limpa.

In [ ]:
%%bash
cd recommendation-frequency
mvn package -q
echo "JAR compilado com sucesso em: target/recommendation-frequency-1.0-SNAPSHOT.jar"

## 2. Ingestão de Dados com Sqoop

Agora, vamos importar os dados de agendamentos do PostgreSQL para o HDFS. O comando `hdfs dfs -rm` é usado para remover o diretório de destino antes da importação, garantindo que possamos executar esta célula várias vezes sem erros.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_agendamentos

# Remove o diretório de entrada se ele já existir
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

echo "Iniciando importação com Sqoop..."
sqoop import \
    --connect jdbc:postgresql://localhost:5432/postgres \
    --username postgres \
    --password postgres \
    --query "SELECT pet.pet_id, pet.nome, agendamento.data_agendamento FROM agendamento JOIN pet ON agendamento.pet_id = pet.pet_id WHERE agendamento.status = 'Realizado' AND \$CONDITIONS" \
    --target-dir $INPUT_DIR \
    --m 1 \
    --split-by pet.pet_id

echo "\nImportação concluída. Verificando os dados no HDFS:"
hdfs dfs -ls $INPUT_DIR
hdfs dfs -cat $INPUT_DIR/part-m-00000 | head -n 5

## 3. Execução do Job MapReduce

Com os dados no HDFS, podemos executar nosso job MapReduce. A célula abaixo submete o `.jar` que compilamos para o Hadoop. O resultado será salvo no diretório `/petshop/output_recomendacoes`.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_agendamentos
OUTPUT_DIR=/petshop/output_recomendacoes
JAR_PATH=recommendation-frequency/target/recommendation-frequency-1.0-SNAPSHOT.jar

# Remove o diretório de entrada se ele já existir (para evitar FileAlreadyExistsException se for usado como output)
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de entrada existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

# Remove o diretório de saída se ele já existir
hdfs dfs -test -d $OUTPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de saída existente: $OUTPUT_DIR"
    hdfs dfs -rm -r $OUTPUT_DIR
fi

echo "Executando o job MapReduce..."
hadoop jar $JAR_PATH com.petshop.hadoop.FrequencyRecommendation $INPUT_DIR $OUTPUT_DIR

echo "\nJob concluído!"

## 4. Verificação do Resultado

Vamos verificar o resultado processado no HDFS. O arquivo `part-r-00000` deve conter o `pet_id`, a `data_sugerida` e a `frequencia_media`.

In [ ]:
%%bash
OUTPUT_DIR=/petshop/output_recomendacoes
echo "Conteúdo do diretório de saída:"
hdfs dfs -ls $OUTPUT_DIR

echo "\nResultado do processamento:"
hdfs dfs -cat $OUTPUT_DIR/part-r-00000

## 5. Carregando e Consultando no Redis (Exemplo)

A etapa final na arquitetura real seria carregar este resultado no Redis para consulta rápida. O serviço `loader-py` faria isso automaticamente. Abaixo, simulamos como os dados seriam armazenados e consultados usando `redis-cli`.

In [ ]:
%%bash
# O nome 'redis' funciona porque os contêineres estão na mesma rede Docker
redis-cli -h redis HSET recomendacao:pet:1 pet_nome "Bidu" data_sugerida "2025-07-13" frequencia_media_dias "31"
redis-cli -h redis EXPIRE recomendacao:pet:1 604800

echo "Dados de recomendação para o pet 1 armazenados no Redis."
echo "Consultando os dados:"
redis-cli -h redis HGETALL recomendacao:pet:1